In [1]:
from ultralytics import YOLO
import ultralytics
import os
import numpy as np
from pathlib import Path
#set visible cuda

os.environ['CUDA_VISIBLE_DEVICES'] = '1'


# Load a pretrained YOLOv8n model
model = YOLO('/Data4/student_zhihan_data/source_code/yolo/ultralytics/runs/detect/GC10-DET_brightness_0 detect by yolov8n with dropout(p=0.1)/weights/best.pt')

# Define path to directory containing images and videos for inference
source = '/Data4/student_zhihan_data/data/GC10-DET/test/images'


# Run inference on the source
results = model([os.path.join(source, i) for i in os.listdir(source)])# generator of Results objects


0: 640x640 1 crescent_gap, 2 welding_lines, 1: 640x640 1 water_spot, 2: 640x640 2 oil_spots, 3: 640x640 1 water_spot, 4: 640x640 4 crescent_gaps, 1 silk_spot, 1 welding_line, 5: 640x640 1 silk_spot, 6: 640x640 3 oil_spots, 3 punching_holes, 1 waist_folding, 7: 640x640 8 water_spots, 8: 640x640 2 waist_foldings, 9: 640x640 1 waist_folding, 10: 640x640 1 crescent_gap, 2 welding_lines, 11: 640x640 1 crescent_gap, 12: 640x640 (no detections), 13: 640x640 1 silk_spot, 14: 640x640 (no detections), 15: 640x640 2 creases, 16: 640x640 1 oil_spot, 17: 640x640 2 water_spots, 18: 640x640 4 oil_spots, 19: 640x640 1 crescent_gap, 1 welding_line, 20: 640x640 3 crescent_gaps, 1 welding_line, 21: 640x640 (no detections), 22: 640x640 1 punching_hole, 23: 640x640 3 welding_lines, 24: 640x640 2 punching_holes, 25: 640x640 1 silk_spot, 26: 640x640 2 punching_holes, 1 welding_line, 27: 640x640 1 water_spot, 28: 640x640 1 water_spot, 29: 640x640 (no detections), 30: 640x640 1 water_spot, 31: 640x640 1 water

In [2]:
from ultralytics.utils.metrics import compute_ap
from ultralytics.engine.validator import BaseValidator
from ultralytics.utils.metrics import box_iou, Metric, DetMetrics
import numpy as np
import torch
import pandas as pd

def smooth(y, f=0.05):
    """Box filter of fraction f."""
    nf = round(len(y) * f * 2) // 2 + 1  # number of filter elements (must be odd)
    p = np.ones(nf // 2)  # ones padding
    yp = np.concatenate((p * y[0], y, p * y[-1]), 0)  # y padded
    return np.convolve(yp, np.ones(nf) / nf, mode="valid")  # y-smoothed

def ap_per_class(
    tp, conf, pred_cls, target_cls, plot=False, on_plot=None, save_dir=Path(), names=(), eps=1e-16, prefix=""
):
    """
    Computes the average precision per class for object detection evaluation.

    Args:
        tp (np.ndarray): Binary array indicating whether the detection is correct (True) or not (False).
        conf (np.ndarray): Array of confidence scores of the detections.
        pred_cls (np.ndarray): Array of predicted classes of the detections.
        target_cls (np.ndarray): Array of true classes of the detections.
        plot (bool, optional): Whether to plot PR curves or not. Defaults to False.
        on_plot (func, optional): A callback to pass plots path and data when they are rendered. Defaults to None.
        save_dir (Path, optional): Directory to save the PR curves. Defaults to an empty path.
        names (tuple, optional): Tuple of class names to plot PR curves. Defaults to an empty tuple.
        eps (float, optional): A small value to avoid division by zero. Defaults to 1e-16.
        prefix (str, optional): A prefix string for saving the plot files. Defaults to an empty string.

    Returns:
        (tuple): A tuple of six arrays and one array of unique classes, where:
            tp (np.ndarray): True positive counts at threshold given by max F1 metric for each class.Shape: (nc,).
            fp (np.ndarray): False positive counts at threshold given by max F1 metric for each class. Shape: (nc,).
            p (np.ndarray): Precision values at threshold given by max F1 metric for each class. Shape: (nc,).
            r (np.ndarray): Recall values at threshold given by max F1 metric for each class. Shape: (nc,).
            f1 (np.ndarray): F1-score values at threshold given by max F1 metric for each class. Shape: (nc,).
            ap (np.ndarray): Average precision for each class at different IoU thresholds. Shape: (nc, 10).
            unique_classes (np.ndarray): An array of unique classes that have data. Shape: (nc,).
            p_curve (np.ndarray): Precision curves for each class. Shape: (nc, 1000).
            r_curve (np.ndarray): Recall curves for each class. Shape: (nc, 1000).
            f1_curve (np.ndarray): F1-score curves for each class. Shape: (nc, 1000).
            x (np.ndarray): X-axis values for the curves. Shape: (1000,).
            prec_values: Precision values at mAP@0.5 for each class. Shape: (nc, 1000).
    """

    # Sort by objectness
    i = np.argsort(-conf)
    tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]

    # Find unique classes
    unique_classes, nt = np.unique(target_cls, return_counts=True)
    nc = unique_classes.shape[0]  # number of classes, number of detections

    # Create Precision-Recall curve and compute AP for each class
    x, prec_values = np.linspace(0, 1, 1000), []

    # Average precision, precision and recall curves
    ap, p_curve, r_curve = np.zeros((nc, tp.shape[1])), np.zeros((nc, 1000)), np.zeros((nc, 1000))
    for ci, c in enumerate(unique_classes):
        i = pred_cls == c
        n_l = nt[ci]  # number of labels
        n_p = i.sum()  # number of predictions
        if n_p == 0 or n_l == 0:
            continue

        # Accumulate FPs and TPs
        fpc = (1 - tp[i]).cumsum(0)
        tpc = tp[i].cumsum(0)

        # Recall
        recall = tpc / (n_l + eps)  # recall curve
        r_curve[ci] = np.interp(-x, -conf[i], recall[:, 0], left=0)  # negative x, xp because xp decreases

        # Precision
        precision = tpc / (tpc + fpc)  # precision curve
        p_curve[ci] = np.interp(-x, -conf[i], precision[:, 0], left=1)  # p at pr_score

        # AP from recall-precision curve
        for j in range(tp.shape[1]):
            ap[ci, j], mpre, mrec = compute_ap(recall[:, j], precision[:, j])
            if plot and j == 0:
                prec_values.append(np.interp(x, mrec, mpre))  # precision at mAP@0.5

    prec_values = np.array(prec_values)  # (nc, 1000)

    # Compute F1 (harmonic mean of precision and recall)
    f1_curve = 2 * p_curve * r_curve / (p_curve + r_curve + eps)
    f2_curve = (5 * p_curve * r_curve) / (4 * p_curve + r_curve + eps)
    # names = [v for k, v in names.items() if k in unique_classes]  # list: only classes that have data
    # names = dict(enumerate(names))  # to dict
    # if plot:
    #     plot_pr_curve(x, prec_values, ap, save_dir / f"{prefix}PR_curve.png", names, on_plot=on_plot)
    #     plot_mc_curve(x, f1_curve, save_dir / f"{prefix}F1_curve.png", names, ylabel="F1", on_plot=on_plot)
    #     plot_mc_curve(x, p_curve, save_dir / f"{prefix}P_curve.png", names, ylabel="Precision", on_plot=on_plot)
    #     plot_mc_curve(x, r_curve, save_dir / f"{prefix}R_curve.png", names, ylabel="Recall", on_plot=on_plot)

    i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
    p, r, f1, f2 = p_curve[:, i], r_curve[:, i], f1_curve[:, i], f2_curve[:, i]  # max-F1 precision, recall, F1 values
    tp = (r * nt).round()  # true positives
    fp = (tp / (p + eps) - tp).round()  # false positives
    return tp, fp, p, r, f1, f2, ap, unique_classes.astype(int), p_curve, r_curve, f1_curve, f2_curve, x, prec_values


In [ ]:
def compute_f2(source, model):
    results = model([os.path.join(source, i) for i in os.listdir(source)])
    validator = BaseValidator()
    validator.iouv = torch.arange(0.5, 1, 0.05)
    
    df = pd.DataFrame(columns=['img_name', 'p', 'r', 'f1', 'f2', 'conf', 'pred_cls', 'target_cls'])

    for result in results:
        metric = DetMetrics()
        label_path = result.path.replace('images', 'labels')[:-4] + '.txt'
        label = torch.from_numpy(np.loadtxt(label_path))
        # turn label into (x1, y1, x2, y2) format
        
        if len(label) == 0:
            continue
        
        if label.dim() > 1:
            x1 = label[:, 1] - label[:, 3] / 2
            y1 = label[:, 2] - label[:, 4] / 2
            x2 = label[:, 1] + label[:, 3] / 2
            y2 = label[:, 2] + label[:, 4] / 2
            label = torch.stack((label[:, 0], x1, y1, x2, y2), 1)
            iou = box_iou(label[:, 1:].to("cuda:0"), result.boxes.xyxyn.to("cuda:0"))
            tp = validator.match_predictions(result.boxes.data[:,-1], label[:, 0].to("cuda:0"), iou)
            
            # update metric
            tp = tp.detach().cpu().numpy()
            conf = result.boxes.conf.detach().cpu().numpy()
            pre_cls = result.boxes.cls.detach().cpu().numpy()
            target_cls = label[:, 0].detach().cpu().numpy()
        
        elif label.dim() == 1:
            x1 = label[1] - label[3] / 2
            y1 = label[2] - label[4] / 2
            x2 = label[1] + label[3] / 2
            y2 = label[2] + label[4] / 2
            label = torch.tensor([label[0], x1, y1, x2, y2])
            iou = box_iou(label[1:].to("cuda:0").reshape(1, -1), result.boxes.xyxyn.to("cuda:0"))
            tp = validator.match_predictions(result.boxes.data[:,-1], label[0].to("cuda:0").unsqueeze(0), iou)
        
            # update metric
            tp = tp.detach().cpu().numpy()
            conf = result.boxes.conf.detach().cpu().numpy()
            pre_cls = result.boxes.cls.detach().cpu().numpy()
            target_cls = label[0].unsqueeze(0).detach().cpu().numpy()
            
        _, _, p, r, f1, f2, ap, unique_classes, p_curve, r_curve, f1_curve, f2_curve, x, prec_values = ap_per_class(tp, conf, pre_cls, target_cls)
        print(f2.mean(), unique_classes)
        
        # update df
        df.loc[len(df)] = [result.path, p, r, f1, f2, conf, pre_cls, target_cls]
        
    df.to_csv(f'/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/F2_Record/{source.split("/")[-3]}.csv', index=False, header=True)
    return df
  

# for i in os.listdir('/Data4/student_zhihan_data/data'):
#     if i[-3:] != 'csv' and i != 'NEU-DET' and i != 'data.zip':
#         source = os.path.join('/Data4/student_zhihan_data/data', i, 'test/images')
#         df = compute_f2(source, model)

In [ ]:
# input: uncertainty * 3, proposed * score, NIQE, BRISQUE
# output: quality score 

F2_dir = '/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/F2_Record'
Proposed_dir = '/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Proposed_Score_Record'
Uncertainty_dir = '/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Uncertainty_Record'

# concat all csv
dfs = []
for idx, dir in enumerate([F2_dir, Proposed_dir, Uncertainty_dir]):
    files = os.listdir(dir)
    df = pd.DataFrame()
    for file in files:
        if file[-3:] == 'csv':
            df = pd.concat([df, pd.read_csv(os.path.join(dir, file))], ignore_index=True)
    if idx == 0:       
        df.to_csv(f'{dir}.csv', index=False, header=True)
    else:
        df['img_name'] = df['dataset'] + '/images/' + df['img_name']
        df = df.drop(columns=['dataset'])
        df.to_csv(f'{dir}.csv', index=False, header=True)
    dfs.append(df)



In [30]:
files = os.listdir(Proposed_dir)
df = pd.DataFrame(columns=['dataset', 'img_name', 'visibility', 'exposure'])
for file in files:
    if file[-3:] == 'csv':
        tmp = pd.read_csv(os.path.join(Proposed_dir, file))
        #add header to tmp
        tmp.columns = ['dataset', 'img_name', 'visibility', 'exposure']
        df = pd.concat([df, tmp], ignore_index=True)

df.to_csv(f'Proposed_Record.csv', index=False, header=True)

/tmp/ipykernel_2347512/2492081121.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, tmp], ignore_index=True)


In [99]:
dfs = []
for idx, dir in enumerate(['/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/F2_Record.csv',
                          '/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Proposed_Record.csv',
                           '/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Uncertainty_Record.csv']):
    df = pd.read_csv(dir)
    dfs.append(df)
 
merged_df = pd.merge(dfs[0], dfs[1], on=['img_name'])
merged_df = pd.merge(merged_df, dfs[2], on=['img_name'])
    
    
# columns = set()
# for df in dfs:
#     columns = columns.union(set(df.columns))

# df = pd.DataFrame(columns=list(columns))
# img_name_union = set(dfs[0]['img_name'])

# for tmp in dfs:
#     img_name_union = img_name_union.intersection(set(tmp['img_name']))

# for tmp in dfs:
#     merged_df = pd.merge(df, tmp, on=['img_name'])
    

In [101]:
merged_df.to_csv('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/merged.csv', index=False, header=True)

In [94]:
df = pd.read_csv('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Uncertainty_Record.csv')
a =  df['img_name'].apply(lambda x: '/'.join(x.split('/')[:-2]))
b = df['img_name'].apply(lambda x: '/'.join(x.split('/')[-2:]))

#concat a, '/images/, b to a new column
df['img_name'] = a + '/test/' + b
df.to_csv('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Uncertainty_Record.csv', index=False, header=True)

In [61]:
df = pd.read_csv('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Proposed_Record.csv')
df['img_name'] = df['dataset'] + '/images/' + df['img_name']
df = df.drop(columns=['dataset'])
df.to_csv('/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/Proposed_Record.csv', index=False, header=True)

In [ ]:
from ultralytics.engine.validator import BaseValidator
from ultralytics.utils.metrics import box_iou, Metric, DetMetrics
import numpy as np
import torch
import pandas as pd

# load txt label


validator = BaseValidator()
validator.iouv = torch.arange(0.5, 1, 0.05)

for result in results:
    metric = DetMetrics()
    label_path = result.path.replace('images', 'labels')[:-4] + '.txt'
    label = torch.from_numpy(np.loadtxt(label_path))
    # turn label into (x1, y1, x2, y2) format
    
    if label.dim() > 1:
        x1 = label[:, 1] - label[:, 3] / 2
        y1 = label[:, 2] - label[:, 4] / 2
        x2 = label[:, 1] + label[:, 3] / 2
        y2 = label[:, 2] + label[:, 4] / 2
        label = torch.stack((label[:, 0], x1, y1, x2, y2), 1)
        iou = box_iou(label[:, 1:].to("cuda:0"), result.boxes.xyxyn.to("cuda:0"))
        tp = validator.match_predictions(result.boxes.data[:,-1], label[:, 0].to("cuda:0"), iou)
        
        # update metric
        tp = tp.detach().cpu().numpy()
        conf = result.boxes.conf.detach().cpu().numpy()
        pre_cls = result.boxes.cls.detach().cpu().numpy()
        target_cls = label[:, 0].detach().cpu().numpy()
    
    else:
        x1 = label[1] - label[3] / 2
        y1 = label[2] - label[4] / 2
        x2 = label[1] + label[3] / 2
        y2 = label[2] + label[4] / 2
        label = torch.tensor([label[0], x1, y1, x2, y2])
        iou = box_iou(label[1:].to("cuda:0").reshape(1, -1), result.boxes.xyxyn.to("cuda:0"))
        tp = validator.match_predictions(result.boxes.data[:,-1], label[0].to("cuda:0").unsqueeze(0), iou)
    
        # update metric
        tp = tp.detach().cpu().numpy()
        conf = result.boxes.conf.detach().cpu().numpy()
        pre_cls = result.boxes.cls.detach().cpu().numpy()
        target_cls = label[0].unsqueeze(0).detach().cpu().numpy()
    
    _, _, p, r, f1, f2, ap, unique_classes, p_curve, r_curve, f1_curve, f2_curve, x, prec_values = ap_per_class(tp, conf, pre_cls, target_cls)
    print(f2.mean(), unique_classes)
    
      

In [ ]:
# Process results generator
for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs


In [106]:
import numpy as np
import ast
import pandas as pd

def safe_convert_to_list(s):
    try:
        # Attempt to directly evaluate the string
        return ast.literal_eval(s)
    except SyntaxError:
        # If direct evaluation fails, attempt to manually parse the string
        cleaned_str = s.strip('[]')
        if cleaned_str:  # Check if the string is not empty
            numbers = [float(num) for num in cleaned_str.split() if num not in ['[', ']']]
            return numbers
        else:
            return []


# Load the dataset
data_path = '/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/merged.csv'
data = pd.read_csv(data_path)

# Display the first few rows of the dataframe to understand its structure
data.head()

# Apply the conversion function to the 'f2' column and then compute the mean F2 score
data['f2'] = data['f2'].apply(safe_convert_to_list)
data['mean_f2'] = data['f2'].apply(lambda x: np.mean(x) if len(x) > 0 else np.nan)

# Prepare the dataset for modeling
features = ['visibility', 'exposure', 'objectness_uncertainty', 'weighted_variance_sum', 'weighted_entropy']
X = data[features]
y = data['mean_f2']

# Display the first few rows of features and target to verify

,visibility,exposure,objectness_uncertainty,weighted_variance_sum,weighted_entropy
0,0.533106,0.034241,0.003777,0.158915,0.000058
1,0.694677,0.001499,0.009511,0.120407,0.000020
2,0.354166,0.152297,0.009658,0.061321,0.000055
3,0.006219,0.113457,0.016292,0.082116,0.000533
4,0.165364,0.105627,0.002982,0.091819,0.000116


In [117]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree Regressor": DecisionTreeRegressor(random_state=42),
    "MLP Regressor": MLPRegressor(random_state=42, max_iter=1000), # Increased max_iter for convergence
    "Random Forest Regressor": RandomForestRegressor(random_state=42),
}

# Train and evaluate each model
results = {}
for name, model in models.items():
    # Train
    model.fit(X_train, y_train)
    # Predict
    y_pred = model.predict(X_test)
    # Evaluate
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {"MSE": mse, "R2": r2}
    
    if name == 'Decision Tree Regressor':
        # save test resulst to csv
        df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
        df.to_csv(f'/Data4/student_zhihan_data/source_code/IQA_A-STAR/source_code/Mydemo/test_result.csv', index=False)

results_df = pd.DataFrame(results).T  # Convert results to a DataFrame for better readability
results_df

,MSE,R2
Linear Regression,0.188070,0.107889
Decision Tree Regressor,0.275943,-0.308935
MLP Regressor,0.185389,0.120607
Random Forest Regressor,0.140655,0.332804


In [119]:
model.feature_importances_

array([    0.16305,     0.16094,     0.16593,      0.2284,     0.28168])

In [112]:
X_train.describe()

,visibility,exposure,objectness_uncertainty,weighted_variance_sum,weighted_entropy
count,4.867000e+03,4867.000000,4.867000e+03,4867.000000,4867.000000
mean,2.796616e-01,0.232434,6.412672e-03,0.144884,0.001363
std,2.144880e-01,0.268423,4.923872e-03,0.072237,0.004153
min,3.222425e-07,0.000000,1.274079e-07,0.050507,0.000000
25%,1.034640e-01,0.016136,2.396813e-03,0.096459,0.000037
50%,2.356180e-01,0.119043,5.491689e-03,0.105285,0.000154
75%,4.143271e-01,0.384620,9.171929e-03,0.203721,0.000790
max,9.695124e-01,0.999207,2.989884e-02,0.479202,0.056565


In [113]:
X_test.describe()

,visibility,exposure,objectness_uncertainty,weighted_variance_sum,weighted_entropy
count,1217.000000,1217.000000,1.217000e+03,1217.000000,1217.000000
mean,0.270493,0.230253,6.240577e-03,0.149462,0.001661
std,0.209810,0.272485,4.841622e-03,0.073080,0.004813
min,0.000420,0.000000,3.902979e-08,0.055160,0.000000
25%,0.093177,0.010945,2.279311e-03,0.097146,0.000037
50%,0.230777,0.109753,5.438593e-03,0.107024,0.000173
75%,0.402377,0.393562,8.849673e-03,0.226471,0.000954
max,0.958951,0.999026,2.272954e-02,0.413786,0.040970


In [114]:
y_train.describe()

count    4867.000000
mean        0.476630
std         0.462209
min         0.000000
25%         0.000000
50%         0.500000
75%         1.000000
max         1.000000
Name: mean_f2, dtype: float64

In [116]:
y_test.describe()

count    1217.000000
mean        0.474405
std         0.459334
min         0.000000
25%         0.000000
50%         0.500000
75%         1.000000
max         1.000000
Name: mean_f2, dtype: float64